In [ ]:
%%capture
!pip install ultralytics kaggle pyyaml matplotlib seaborn --quiet

In [ ]:
import os
import yaml
import glob
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import torch
from ultralytics import YOLO
from google.colab import files

In [ ]:
# Set up environment
sns.set_theme(style="darkgrid", font_scale=1.5)
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# Set random seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

In [ ]:
# Download and prepare dataset
from google.colab import files
uploaded = files.upload()

print("Setting up dataset...")
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d muki2003/yolo-drone-detection-dataset
!unzip -q yolo-drone-detection-dataset.zip -d drone_dataset
!rm yolo-drone-detection-dataset.zip

In [ ]:
# Create YAML config
config = {
    "path": "/content/drone_dataset/drone_dataset",
    "train": "/content/drone_dataset/drone_dataset/train/images",
    "val": "/content/drone_dataset/drone_dataset/valid/images",
    "names": ["drone"],
    "nc": 1
}

with open("/content/data.yaml", "w") as f:
    yaml.dump(config, f)

In [ ]:
# Train model
print("\nTraining model...")
os.environ["WANDB_DISABLED"] = "true"  # Disable wandb
model = YOLO("yolov8n.pt")

results = model.train(
    data="/content/data.yaml",
    epochs=50,
    imgsz=640,
    batch=8,
    device=0,
    patience=5,
    seed=42
)

In [ ]:
# Evaluate model
metrics = model.val()
print(f"\nEvaluation results:")
print(f"mAP@0.5: {metrics.box.map:.3f}")

In [ ]:
# Test on sample images
print("\nTesting on sample images...")
test_images = glob.glob("/content/drone_dataset/drone_dataset/valid/images/*.jpg")[:3]
for img_path in test_images:
    results = model.predict(img_path, conf=0.5)
    for r in results:
        plt.figure(dpi=100)
        plt.imshow(r.plot()[:,:,::-1])  # Convert BGR to RGB
        plt.title(f"{len(r.boxes)} objects detected" if len(r.boxes) > 0 else "No objects detected")
        plt.axis("off")
        plt.show()

In [ ]:
# Video Processing Section
print("\nVideo Processing Section")
print("Please upload your video file:")
uploaded = files.upload()
video_name = list(uploaded.keys())[0]
input_path = f"/content/{video_name}"

In [ ]:
# Verify model exists
model_path = "/content/runs/detect/train/weights/best.pt"
if not os.path.exists(model_path):
    raise FileNotFoundError(f"Trained model not found at {model_path}")

In [ ]:
# Load trained model
model = YOLO(model_path)

In [ ]:
# Process video
output_dir = "/content/video_results"
os.makedirs(output_dir, exist_ok=True)

# Define output filename with MP4 extension
output_video_name = os.path.splitext(video_name)[0] + "_output.mp4"
output_video_path = f"{output_dir}/{output_video_name}"

print(f"\nProcessing video: {video_name}...")
results = model.predict(
    source=input_path,
    conf=0.5,
    save=True,
    imgsz=640,
    device=0,
    project=output_dir,
    name="processed_video",
    save_txt=True,
    save_conf=True
)

In [ ]:
# Find the actual output path (Ultralytics changes the output naming)
processed_dir = f"{output_dir}/processed_video"
output_files = [f for f in os.listdir(processed_dir) if f.endswith('.mp4') or f.endswith('.avi')]

if not output_files:
    print("\nError: No output video file found. Available files:")
    !ls -la {processed_dir}
    raise FileNotFoundError("Processed video not found in output directory")

In [ ]:
# Get the first video file found (should be our processed video)
detected_output = f"{processed_dir}/{output_files[0]}"
final_output = f"{output_dir}/{output_video_name}"

In [ ]:
# Convert to MP4 if needed and copy to final location
if detected_output.endswith('.avi'):
    # Convert AVI to MP4
    cap = cv2.VideoCapture(detected_output)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    out = cv2.VideoWriter(final_output, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()
else:
    # Just copy if already MP4
    !cp {detected_output} {final_output}

In [ ]:
# Download the final file
if os.path.exists(final_output):
    print(f"\n✅ Processing complete! File size: {os.path.getsize(final_output)/1024/1024:.2f} MB")
    files.download(final_output)
else:
    print("\nError: Final output file not created. Available files:")
    !ls -la {output_dir}